In [1]:
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
import numpy as np
import math
import pandas as pd

import random
import logging
logging.basicConfig(level=logging.INFO)


def init_spark():
    spark = SparkSession \
            .builder \
            .config("spark.driver.memory", "50g") \
            .appName("Python Spark SQL basic example") \
            .config("spark.some.config.option", "some-value") \
            .getOrCreate()
    return spark

spark = init_spark()

df2 = pd.read_csv("C:/Users/vansh/Desktop/Poject big data/anime.csv")
df = spark.read.text("C:/Users/vansh/Desktop/Poject big data/sample1.csv").rdd
data = df.map(lambda x: x[0].split(','))
header1=data.first()
data=data.filter(lambda x:x !=header1)
data2 = data.groupBy(lambda x : x[1])
data3 = data2.collect()

#-------------------------------------------------------------------------------------------------------------------------------




#all_data = data.map(lambda x:[int(x[1]),[int(x[0]),int(x[2])]]).collect()



utility = {}
Anime_id = []
user_id = []

for user in range(len(data3)):

    utility[int(data3[user][0])] = {}
    #if int(data3[user][0]) not in all_users:
    #    all_users.append(int(i[0]))
    for i in data3[user][1]:

        if int(i[0]) not in user_id:
          user_id.append(int(i[0]))

        if int(i[2]) == -1:
            continue
        utility[int(i[1])][int(i[0])] = int(i[2])


Anime_id= list(utility.keys())




In [2]:
utility

{20: {3: 8,
  5: 6,
  21: 8,
  28: 9,
  34: 9,
  38: 6,
  39: 10,
  41: 10,
  43: 8,
  46: 7,
  50: 10,
  53: 6,
  60: 9,
  61: 7,
  62: 7,
  72: 8,
  79: 9,
  80: 9,
  88: 9,
  95: 7,
  96: 10,
  97: 6,
  99: 5,
  101: 10,
  104: 9,
  108: 6,
  112: 10,
  116: 10,
  118: 7,
  124: 7,
  125: 4,
  128: 8,
  129: 9,
  130: 10,
  132: 9,
  140: 7,
  148: 8,
  153: 6,
  159: 7,
  163: 9,
  164: 9,
  166: 6,
  171: 10,
  173: 10,
  177: 9,
  179: 6,
  185: 7,
  187: 6,
  189: 8,
  190: 9,
  191: 7,
  195: 7,
  196: 7,
  199: 9,
  202: 6,
  209: 9,
  210: 10,
  214: 7,
  219: 9,
  220: 10,
  226: 7,
  234: 5,
  235: 8,
  236: 7,
  241: 8,
  246: 9,
  248: 9,
  249: 8,
  250: 7,
  252: 8,
  253: 6,
  258: 9,
  260: 9,
  261: 7,
  263: 7,
  268: 8,
  269: 7,
  271: 1,
  272: 8,
  273: 5,
  275: 9,
  279: 7,
  281: 8,
  282: 9,
  284: 7,
  288: 8,
  290: 9,
  293: 8,
  294: 10,
  296: 10,
  297: 10,
  298: 9,
  311: 10,
  312: 9,
  316: 9,
  317: 9,
  322: 9,
  325: 8,
  329: 7,
  332: 5,
  334

In [3]:
def sim2(matrix, item1, item2):
    matched = []

    for user in list(matrix[item1].keys()):





        if user in list(matrix[item2].keys()):
            if user not in matched:

                matched.append(user)






    x = []
    y = []

    for user in matched:

            x.append(matrix[item1][int(user)])
            y.append(matrix[item2][int(user)])

    f = (lambda i, j: i - j)

    x = f(x, np.mean(x))

    y = f(y, np.mean(y))

    #Pearson Correlation Coefficient
    numerator = (lambda x, y : x*y)

    denominator = (lambda x: x*x)
    d = math.sqrt(sum(denominator(x))) * math.sqrt(sum(denominator(y)))

    if d == 0:
        return 0
    similarity = sum(numerator(x, y))/ d
    return similarity




def most_similar_item(matrix, item, n=10):
    similarity = {}
    for p in list(matrix.keys()):


        similarity[p] = sim2(matrix, item, p)
    sorted_list = sorted(similarity.items(), key = lambda kv:(kv[1], kv[0]))
    return sorted_list[-n:]



def recommend(matrix, item):
    similar_items = most_similar_item(matrix, item)


    predictions = {}
    for user in user_id:

        num = 0
        den = 0
        user=int(user)
        for item in similar_items:

            if  user in matrix[item[0]].keys():


                num +=   int(matrix[item[0]][user]) * int(item[1])
                den += int(item[1])


        if den == 0:
            continue
        if user in matrix[item[0]].keys():
            predictions[user] = float(num/den)
    return predictions

test= int(0.05 * len(Anime_id))


test_data_item = []
for _ in range(50):
    random_item = random.randint(1,  2000)
    if Anime_id[random_item] not in test_data_item:
        test_data_item.append(Anime_id[random_item])

test_user = int(0.1 * len(user_id))
test_data_user = []
for _ in range(test_user):
    random_user = random.randint(1, len(user_id)-1)
    if user_id[random_user] not in test_data_user:
        test_data_user.append(user_id[random_user])


import copy
training_data = copy.deepcopy(utility)

for anime in test_data_item:
    for user in test_data_user:
        user=int(user)
        if (user) in training_data[anime].keys():

            del training_data[anime][user]


def start():
    predicted = {}

    for item in test_data_item:

        predicted[item]=recommend(training_data, item)


    return predicted



predicted = start()



def RMSE(actual, predicted):
    error = 0
    n = 0
    for anime in test_data_item:

        for user in test_data_user:

            if user in list(predicted[anime].keys()) and user in list(actual[anime].keys()):

                error += (int(actual[anime][user]) - int(predicted[anime][user]))**2
                n += 1

    return math.sqrt(error/n)
print(RMSE(utility, predicted))

C:\Users\vansh\Miniconda3\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\vansh\Miniconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


1.0


In [4]:
predictions =[]
def recommend_users(for_animde_id,n=10):
    return recommend(training_data,for_animde_id)
for anime_id in Anime_id:

      predictions.append([anime_id,recommend_users(anime_id)])

print(predictions)

final=[]
final_predictions={}

[[20, {392: 5.0, 418: 7.666666666666667, 462: 6.0}], [24, {5: 2.2, 226: 8.0, 271: 7.333333333333333, 282: 8.8, 379: 7.25, 404: 7.0, 405: 7.666666666666667, 443: 10.0, 492: 4.571428571428571, 497: 9.0, 120: 9.0, 256: 8.333333333333334, 321: 7.833333333333333, 446: 6.25, 17: 6.5, 326: 9.0, 7: 7.166666666666667}], [79, {53: 8.5, 79: 10.0, 80: 10.0, 177: 8.0, 191: 8.0, 236: 7.0, 250: 10.0, 271: 5.75, 282: 9.0, 293: 8.0, 294: 8.0, 354: 8.0, 392: 5.25, 395: 8.0, 478: 8.428571428571429, 183: 6.5, 186: 9.0, 438: 8.0, 256: 9.166666666666666, 461: 8.0, 37: 9.0, 155: 8.0, 244: 8.666666666666666, 326: 10.0, 348: 5.0, 500: 9.0, 301: 7.333333333333333, 373: 7.5, 233: 10.0, 87: 9.0, 350: 10.0, 428: 9.0}], [226, {271: 7.666666666666667, 17: 6.6, 462: 7.0, 123: 7.333333333333333}], [241, {5: 4.0, 41: 9.0, 317: 6.5, 339: 8.0, 492: 6.333333333333333, 320: 9.0, 321: 9.0, 446: 9.0, 17: 5.0, 341: 7.5, 371: 8.0, 7: 7.25, 433: 9.0}], [355, {43: 7.333333333333333, 177: 7.0, 271: 8.5, 322: 7.0, 325: 6.0, 392: 5

In [7]:
final={}
for user in user_id:
    for it in predictions:
        if int(user) in it[1].keys():
            final[it[0]]=int(it[1].get((user)))
            final={k: final[k] for k in sorted(final, key=final.get, reverse=True)}

    final_predictions[user]=final
    final={}



print(final_predictions)


{1: {12361: 10, 15847: 10, 6920: 10, 10260: 10, 1655: 10, 461: 10, 3323: 10, 109: 10, 10380: 10, 3247: 10, 2391: 10, 2580: 10}, 3: {4581: 10, 18097: 10, 534: 10, 15335: 10, 2026: 10, 25687: 10, 570: 10, 10092: 10, 24151: 10, 24893: 10, 985: 10, 2775: 10, 6895: 10, 13115: 10, 878: 10, 1023: 10, 6444: 10, 9982: 10, 6401: 10, 395: 10, 1569: 10, 1842: 10, 3467: 10, 8394: 10, 16099: 10, 10715: 10, 7588: 10, 1065: 10, 130: 10, 131: 10, 1110: 10, 2142: 10, 20359: 10, 17505: 9, 138: 9, 508: 9, 550: 9, 6974: 9, 13201: 9, 8795: 9, 22135: 9, 22839: 9, 2156: 9, 169: 9, 856: 9, 9065: 9, 6920: 9, 10260: 9, 9745: 9, 11837: 9, 10379: 9, 20673: 9, 2390: 9, 634: 9, 3226: 9, 8426: 9, 10647: 9, 13709: 9, 19251: 9, 3323: 9, 8624: 9, 10278: 9, 10380: 9, 537: 9, 10017: 9, 2391: 9, 11266: 8, 1121: 8, 1122: 8, 14093: 8, 21507: 8, 137: 8, 248: 8, 898: 8, 3887: 8, 4472: 8, 8457: 8, 11371: 8, 13535: 8, 24781: 8, 30347: 8, 338: 8, 1709: 8, 2685: 8, 437: 8, 9834: 8, 133: 8, 4938: 8, 10624: 8, 12029: 8, 19489: 8, 12

In [8]:
def final_recommendation( user):
    anime=final_predictions.get(user).keys()
    for item in anime:
        values = df2[['anime_id', 'name', 'genre', 'type']].loc[df2['anime_id'] == item]
        print(str(values['anime_id']) + "," + str(values['name'] + "," + str(values['genre']) + "," + str(values['type'] + "\n")))


final_recommendation(3)



2019    4581
Name: anime_id, dtype: int64,2019    Shikabane Hime: Aka,2019    Action, Horror, Ma...
Name: name, dtype: object
2946    18097
Name: anime_id, dtype: int64,2946    Inazuma Eleven Go: Galaxy,2946    Shounen, Spo...
Name: name, dtype: object
736    534
Name: anime_id, dtype: int64,736    Slayers,736    Adventure, Comedy, Demons, Fant...
Name: name, dtype: object
8    15335
Name: anime_id, dtype: int64,8    Gintama Movie: Kanketsu-hen - Yorozuya yo Eien...
Name: name, dtype: object
951    2026
Name: anime_id, dtype: int64,951    Hayate no Gotoku!,951    Action, Comedy, Harem...
Name: name, dtype: object
1135    25687
Name: anime_id, dtype: int64,1135    Digimon Adventure tri. 1: Saikai,1135    Actio...
Name: name, dtype: object
726    570
Name: anime_id, dtype: int64,726    Jin-Rou,726    Drama, Military, Police, Psycho...
Name: name, dtype: object
809    10092
Name: anime_id, dtype: int64,809    Break Blade 6: Doukoku no Toride,809    Action...
Name: name, dtype: object
924 